In [3]:
import numpy as np
import os
import glob
from math import floor
import random
import time
import cv2

In [4]:
# check data with boundary
def check_data(Y):
    count_mixed = 0
    count_all_true = 0
    count_all_false = 0
    for i in range(Y.shape[0]):
        if np.sum(Y[i]) == 224 * 224:
            count_all_false += 1
        elif np.sum(Y[i]) == 0:
            count_all_true += 1
        else:
            count_mixed += 1
    print('--- # of all real images is ' + str(count_all_true))
    print('--- # of all forged images is ' + str(count_all_false))
    print('--- # of mixed images is ' + str(count_mixed))

In [5]:
# add forgery to 
import os
import glob
from math import floor
import random
import time
import cv2

def read_image_cm_data(X_train, Y_train, row_size, col_size):
    for filename in glob.glob('*copy.png'):
        prefix = filename[0:-9]
        X_fake = cv2.imread(prefix+'_copy.png') / 255 * 2 - 1
        X_true = cv2.imread(prefix+'.png') / 255 * 2 - 1
        
        Y_fake = np.expand_dims(cv2.imread(prefix+'_1_alpha.png', 0), axis = -1) / 255
        if os.path.exists(prefix+'_2_alpha.png'):
            Y_fake = np.logical_or(Y_fake, np.expand_dims(cv2.imread(prefix+'_2_alpha.png', 0), axis = -1) / 255)
        if os.path.exists(prefix+'_3_alpha.png'):
            Y_fake = np.logical_or(Y_fake, np.expand_dims(cv2.imread(prefix+'_3_alpha.png', 0), axis = -1) / 255)
        if os.path.exists(prefix+'_4_alpha.png'):
            Y_fake = np.logical_or(Y_fake, np.expand_dims(cv2.imread(prefix+'_4_alpha.png', 0), axis = -1) / 255)

        num_row_split = floor(X_fake.shape[0] / row_size) # round down
        num_col_split = floor(X_fake.shape[1] / col_size) # round down
        for i in range(num_row_split):
            for j in range(num_col_split):
                splitted_image_X_fake = X_fake[i*row_size:(i+1)*row_size, j*col_size:(j+1)*col_size, :]
                splitted_image_Y_fake = Y_fake[i*row_size:(i+1)*row_size, j*col_size:(j+1)*col_size, :]
                splitted_image_X_true = X_true[i*row_size:(i+1)*row_size, j*col_size:(j+1)*col_size, :]
                X_train.append(splitted_image_X_fake)
                #X_train.append(splitted_image_X_true)
                Y_train.append(splitted_image_Y_fake)
                #Y_train.append(np.ones((row_size, col_size,1)))
def weaken_RGB(RGB, original_image, x, y, w, h, coeffi):
    weaken_image = original_image
    weaken_Y     = np.zeros((original_image.shape[0], original_image.shape[1], 1))
    for i in range(w):
        for j in range(h):
            weaken_Y[y+j, x+i,:] = 1.0
            if RGB < 4:
                weaken_image[y+j, x+i,RGB-1] = weaken_image[y+j, x+i,RGB-1] * coeffi
            elif RGB == 4:
                weaken_image[y+j, x+i,0] = weaken_image[y+j, x+i,0] * coeffi
                weaken_image[y+j, x+i,1] = weaken_image[y+j, x+i,1] * coeffi
            elif RGB == 5:
                weaken_image[y+j, x+i,0] = weaken_image[y+j, x+i,0] * coeffi
                weaken_image[y+j, x+i,2] = weaken_image[y+j, x+i,2] * coeffi
            elif RGB == 6:
                weaken_image[y+j, x+i,1] = weaken_image[y+j, x+i,1] * coeffi
                weaken_image[y+j, x+i,2] = weaken_image[y+j, x+i,2] * coeffi
            elif RGB == 7:
                weaken_image[y+j, x+i,0] = weaken_image[y+j, x+i,0] * coeffi
                weaken_image[y+j, x+i,1] = weaken_image[y+j, x+i,1] * coeffi
                weaken_image[y+j, x+i,2] = weaken_image[y+j, x+i,2] * coeffi
    return weaken_image, weaken_Y
                
def read_image_dresden_data(X_train, Y_train, row_size, col_size):
    count = 0
    false_ratio = []
    for filename in glob.glob('*.jpg'):
        if len(X_train) >= 10000:
            break
        start = time.time()
        count += 1
        if count % 100 == 0:
            print(count)
        X_true = cv2.imread(filename) / 255 * 2 - 1
        coord1 = random.random() * 0.7
        coord2 = random.random() * 0.7
        width  = 1.1
        height = 1.1
        must_exit = False
        while (coord1 + width > 0.99):
            end = time.time()
            if (end-start) > 10:
                print(filename)
                must_exit = True
                break
            width = random.random() * 0.1 +0.28  # 0.07 ~ 0.1
            
        while (coord2 + height > 0.99):
            end = time.time()
            if (end-start) > 10:
                print(filename)
                must_exit = True
                break
            height = random.random() * 0.1 +0.28  # 0.07 ~ 0.1
        if must_exit:
            break
        x = round(coord1 * X_true.shape[1])
        y = round(coord2 * X_true.shape[0])
        w = round(width  * X_true.shape[1])
        h = round(height * X_true.shape[0])
        false_ratio.append(w*h/X_true.shape[1]/X_true.shape[0]);
        
        RGB = random.randint(1,7)
        coeffi = random.random() * 0.7
        X_fake, Y_fake = weaken_RGB(RGB, X_true, x, y, w, h, coeffi)
        
        num_row_split = floor(X_true.shape[0] / row_size) # round down
        num_col_split = floor(X_true.shape[1] / col_size) # round down
        for i in range(num_row_split):
            for j in range(num_col_split):
                splitted_image_X_fake = X_fake[i*row_size:(i+1)*row_size, j*col_size:(j+1)*col_size, :]
                splitted_image_Y_fake = Y_fake[i*row_size:(i+1)*row_size, j*col_size:(j+1)*col_size, :]
                if len(X_train) < 10000:
                    X_train.append(splitted_image_X_fake)
                    Y_train.append(splitted_image_Y_fake)
    print(sum(false_ratio)/len(false_ratio))

In [6]:
row_size = 224
col_size = 224
X_train = []
Y_train = []

In [7]:
os.chdir('/home/ubuntu/cs230_project_shared/benchmark_data_copy_move/C1_panasonic/')
read_image_cm_data(X_train, Y_train, row_size, col_size)
print('1 finish')
os.chdir('/home/ubuntu/cs230_project_shared/benchmark_data_copy_move/C2_canon/')
read_image_cm_data(X_train, Y_train, row_size, col_size)
print('2 finish')
os.chdir('/home/ubuntu/cs230_project_shared/benchmark_data_copy_move/C3_nikon/')
read_image_cm_data(X_train, Y_train, row_size, col_size)
print('3 finish')
os.chdir('/home/ubuntu/cs230_project_shared/benchmark_data_copy_move/C4_flickr/')
read_image_cm_data(X_train, Y_train, row_size, col_size)
print('4 finish')


1 finish
2 finish
3 finish
4 finish


In [8]:
os.chdir('/home/ubuntu/cs230_project_shared/val2017_unsure_type_cocodataset')
read_image_dresden_data(X_train, Y_train, row_size, col_size)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
0.10659527947206612


In [9]:
def train_dev_test_split(X,Y,ratio = [8,1,1]):
    np.random.seed(0)
    r_total = np.sum(ratio)
    ind0 = int(ratio[0]/r_total * X.shape[0])
    ind1 = int((ratio[1] + ratio[0])/r_total * X.shape[0])
    indices = np.random.permutation(X.shape[0])
    train_idx, dev_idx, test_idx = indices[:ind0], indices[ind0:ind1], indices[ind1:]
    x_train, x_dev, x_test = X[train_idx,:], X[dev_idx,:], X[test_idx,:]
    y_train, y_dev, y_test = Y[train_idx,:], Y[dev_idx,:], Y[test_idx,:]
    return x_train, y_train, x_dev, y_dev, x_test, y_test

In [10]:
print(len(X_train))
X_train = np.array(X_train)
Y_train = np.array(Y_train)
print(np.sum(Y_train)/Y_train.shape[0])
print(X_train.shape)


10000
4472.783676862741
(10000, 224, 224, 3)


In [18]:
false_ratio = []
for i in range(len(X_train)):
    if (np.sum(Y_train[i]) == 0):
        if random.random() > 0.025:
            coord1 = random.random() * 0.7
            coord2 = random.random() * 0.7
            width  = 1.1
            height = 1.1
            must_exit = False
            start = time.time()
            while (coord1 + width > 0.99):
                end = time.time()
                if (end-start) > 10:
                    print(filename)
                    must_exit = True
                    break
                width = random.random() * 0.1 +0.28  # 0.07 ~ 0.1
            
            while (coord2 + height > 0.99):
                end = time.time()
                if (end-start) > 10:
                    print(filename)
                    must_exit = True
                    break
                height = random.random() * 0.1 +0.28  # 0.07 ~ 0.1
            if must_exit:
                break
            x = round(coord1 * Y_train[i].shape[1])
            y = round(coord2 * Y_train[i].shape[0])
            w = round(width  * Y_train[i].shape[1])
            h = round(height * Y_train[i].shape[0])
            false_ratio.append(w*h/224/224);
            RGB = random.randint(1,7)
            coeffi = random.random() * 0.7
            X_fake, Y_fake = weaken_RGB(RGB, X_train[i], x, y, w, h, coeffi)
            X_train[i] = X_fake
            Y_train[i] = Y_fake
print(np.sum(Y_train)/Y_train.shape[0])

8163.315217156872


In [19]:
[X_train, Y_train, X_dev, Y_dev, X_test, Y_test] = train_dev_test_split(X_train[:,:,:,:],Y_train[:,:,:,:])\

In [20]:
print('--- check train set ---')
check_data(Y_train)
print(np.sum(Y_train)/Y_train.shape[0])
print('--- check dev set ---')
check_data(Y_dev)
print(np.sum(Y_dev)/Y_dev.shape[0])
print('--- check test set ---')
check_data(Y_test)
print(np.sum(Y_test)/Y_test.shape[0])

--- check train set ---
--- # of all real images is 103
--- # of all forged images is 105
--- # of mixed images is 6192
8093.436319240206
--- check dev set ---
--- # of all real images is 23
--- # of all forged images is 15
--- # of mixed images is 762
8457.038681372549
--- check test set ---
--- # of all real images is 17
--- # of all forged images is 18
--- # of mixed images is 765
8428.622936274505


In [21]:
os.chdir('/home/ubuntu/')
np.save('X_train_2', X_train)
np.save('X_dev_2', X_dev)
np.save('X_test_2', X_test)
np.save('Y_dev_2', Y_dev)
np.save('Y_train_2', Y_train)
np.save('Y_test_2', Y_test)